In [10]:
import numpy as np
import pandas as pd

In [11]:
# Загрузим данные
df = pd.read_csv('E:\\data.csv')

# Преобразуем метки классов в численные значения
class_mapping = {'Iris-setosa': 0, 'Iris-versicolor': 1, 'Iris-virginica': 2}
df['Class'] = df.iloc[:, 4].map(class_mapping)

# Разделим данные на признаки и метки
X = df.iloc[:, :4].values
y = df['Class'].values.reshape(-1, 1)
# Добавим фиктивный признак для удобства матричных вычислений
X = np.concatenate([np.ones((len(X), 1)), X], axis=1)

In [12]:
# Инициализируем нейронную сеть
input_size = X.shape[1]  # Количество входных сигналов равно количеству признаков задачи 
hidden_size = 5  # Задаем количество нейронов в скрытом слое 
output_size = 3  # Количество выходных сигналов равно количеству классов задачи

# Веса инициализируем случайными числами
weights = [
    np.random.uniform(-2, 2, size=(input_size, hidden_size)),  # Веса скрытого слоя
    np.random.uniform(-2, 2, size=(hidden_size, output_size))   # Веса выходного слоя
]

In [13]:
# Зададим функцию активации - сигмоида
def sigmoid(y):
    return 1 / (1 + np.exp(-y))

# Производная от сигмоиды для вычисления градиента
def derivative_sigmoid(y):
    return sigmoid(y) * (1 - sigmoid(y))

In [14]:
# Прямой проход
def feed_forward(x):
    input_ = x
    hidden_ = sigmoid(np.dot(input_, weights[0]))
    output_ = sigmoid(np.dot(hidden_, weights[1]))
    return [input_, hidden_, output_]

# Обратный проход
def backward(learning_rate, target, net_output, layers):
    err = (target - net_output)
    
    for i in range(len(layers) - 1, 0, -1):
        err_delta = err * derivative_sigmoid(layers[i])
        err = np.dot(err_delta, weights[i - 1].T)
        dw = np.dot(layers[i - 1].T, err_delta)
        weights[i - 1] += learning_rate * dw

In [15]:
# Функция обучения
def train(x_values, target, learning_rate):
    output = feed_forward(x_values)
    backward(learning_rate, target, output[2], output)
    return None

# Функция предсказания
def predict(x_values):
    return feed_forward(x_values)[-1]

In [16]:
# Параметры обучения
epochs = 1000
learning_rate = 0.01

# Обучаем сеть
for epoch in range(epochs):
    random_indices = np.arange(len(X))
    np.random.shuffle(random_indices)
    
    X = X[random_indices]
    y = y[random_indices]
    
    for i in range(len(X)):
        x_values = X[i].reshape(1, -1)
        target = np.zeros((1, output_size))
        target[0, y[i]] = 1  # One-hot encoding меток классов
        train(x_values, target, learning_rate)

In [17]:
# Считаем точность на обучающей выборке
correct_predictions = 0
incorrect_predictions = 0

for i in range(len(X)):
    x_values = X[i].reshape(1, -1)
    predicted_class = np.argmax(predict(x_values))
    if predicted_class == y[i]:
        correct_predictions += 1
    else:
        incorrect_predictions += 1

print('correct: ', correct_predictions)
print('incorrect: ', incorrect_predictions)

accuracy = correct_predictions / len(X)
print(f"Точность на обучающей выборке: {accuracy * 100:.2f}%")

correct:  128
incorrect:  22
Точность на обучающей выборке: 85.33%


In [18]:
X = df.iloc[100:101, :4].values
# Добавим фиктивный признак для удобства матричных вычислений
X = np.concatenate([np.ones((len(X), 1)), X], axis=1)
y_pred = predict(X)
y_pred

array([[0.00166384, 0.66894539, 0.99360102]])